Trains and evaluate a simple MLP on the Reuters newswire topic classification task.

Adapted from: https://github.com/fchollet/keras/blob/master/examples/reuters_mlp.py

In [1]:
reset -fs

In [2]:
from keras.datasets import reuters

Using TensorFlow backend.


Dataset of 11,228 newswires from [Reuters](https://www.reuters.com/), labeled over 46 topics. 

For example:

- Business
- Markets
- World
- Politics
- Tech
- Commentary
- Breakingviews
- Money
 - Life

As with the IMDB dataset, each wire is encoded as a sequence of word indexes (same conventions).

In [16]:
print('Loading data...')
num_words = 1_000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=1_000,
                                                        skip_top=25,
                                                        test_split=.2)
print('Data loaded.')

print(f'{len(x_train):,} train sequences')
print(f'{len(x_test):,} test sequences')

Loading data...
Data loaded.
8,982 train sequences
2,246 test sequences


In [17]:
num_classes = max(y_train) + 1 # Assume y takes values 0...K-1 where K is number of classes; Add 1 allow for 1 hot encoding
print(num_classes, 'classes')

46 classes


In [18]:
from keras.preprocessing.text import Tokenizer

In [19]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=1_000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('Data vectorized.')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
Data vectorized.
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [20]:
from keras import utils

In [23]:
print('Convert class vector to binary class matrix (for use with categorical_crossentropy)')
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (19005912, 46)
y_test shape: (4752536, 46)


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [25]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(1_000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print('Model built')

Building model...
Model built


In [29]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)                23598     
_________________________________________________________________
activation_4 (Activation)    (None, 46)                0         
Total params: 536,110.0
Trainable params: 536,110.0
Non-trainable params: 0.0
_________________________________________________________________
None


In [30]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

![](https://cdn-images-1.medium.com/max/1600/1*lKc-R1JFGKVjBkX_wmDQfg.jpeg)

In [31]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=True,
                    validation_split=0.1)

ValueError: Input arrays should have the same number of samples as target arrays. Found 8982 input samples and 19005912 target samples.

In [32]:
score = model.evaluate(x_test, 
                       y_test,
                       batch_size=32, 
                       verbose=True)
print('Test score:', score[0])
print('Test accuracy:', score[1])

ValueError: Input arrays should have the same number of samples as target arrays. Found 2246 input samples and 4752536 target samples.